<a href="https://colab.research.google.com/github/lapshinaaa/homework_machineLearning/blob/main/homework8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашняя работа

В этой работы мы будем учиться предсказывать зарплату data scientist-ов в зависимочти от ряда факторов с помощью градиентоного бустинга

Цель домашней работы научиться предсказывать зарплаты (salary_in_usd) по ряды факторов

work_year: The number of years of work experience in the field of data science.

experience_level: The level of experience, such as Junior, Senior, or Lead.

employment_type: The type of employment, such as Full-time or Contract.

job_title: The specific job title or role, such as Data Analyst or Data Scientist.

salary: The salary amount for the given job.

salary_currency: The currency in which the salary is denoted.

salary_in_usd: The equivalent salary amount converted to US dollars (USD) for comparison purposes.

employee_residence: The country or region where the employee resides.

remote_ratio: The percentage of remote work offered in the job.

company_location: The location of the company or organization.

company_size: The company’s size is categorized as Small, Medium, or Large.

In [34]:
import pandas as pd

In [35]:
!wget https://github.com/hse-ds/iad-intro-ds/raw/master/2023/homeworks/hw-08-boosting/ds_salaries.csv

--2023-06-20 07:50:39--  https://github.com/hse-ds/iad-intro-ds/raw/master/2023/homeworks/hw-08-boosting/ds_salaries.csv
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hse-ds/iad-intro-ds/master/2023/homeworks/hw-08-boosting/ds_salaries.csv [following]
--2023-06-20 07:50:39--  https://raw.githubusercontent.com/hse-ds/iad-intro-ds/master/2023/homeworks/hw-08-boosting/ds_salaries.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 210076 (205K) [text/plain]
Saving to: ‘ds_salaries.csv.4’

ds_salaries.csv.4   100%[===================>] 205.15K  --.-KB/s    in 0.02s   

2023-06-20 07:50:39 (12.

In [69]:
df = pd.read_csv("ds_salaries.csv.4")

In [38]:
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


## Задание 1 (0.5 балла)

Разделите выборку на train, val, test (80%, 10%, 10%) в качестве таргета выберите salary_in_usd, удалите из признаков (salary) чтобы избежать лика в данных.

In [90]:
from sklearn.model_selection import train_test_split

# deleting the 'salary' column, defining X and target variable:
X = df.drop(['salary', 'salary_in_usd'], axis=1)
y = df['salary_in_usd']

# splitting the data into train, val and test^
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=19)
X_val, X_test_test, y_val, y_test_test = train_test_split(X_test, y_test, test_size = 0.5, random_state = 17)

X_val

,work_year,experience_level,employment_type,job_title,salary_currency,employee_residence,remote_ratio,company_location,company_size
3311,2022,EX,FT,Head of Data Science,USD,US,100,US,M
175,2023,SE,FT,Data Analyst,GBP,CF,100,CF,M
3676,2021,SE,FT,Machine Learning Scientist,USD,US,100,CA,L
1604,2023,SE,FT,Machine Learning Engineer,EUR,LV,0,LV,M
1426,2023,SE,FT,Machine Learning Engineer,USD,US,0,US,M
...,...,...,...,...,...,...,...,...,...
3039,2022,SE,FT,Data Science Manager,USD,US,100,US,L
2908,2022,SE,FT,Data Analyst,USD,US,0,US,L
1614,2023,SE,FT,Data Scientist,USD,US,100,US,M
3703,2021,MI,FT,Data Scientist,BRL,BR,0,BR,S


## Задание 2 (0.5 балла)

Обучите модель линейной регресии и оцените её качество через mape и rmse

In [40]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error

categorical_columns = X.select_dtypes(include='object').columns # selecting categorical data
X_train_drop = X_train.drop(categorical_columns, axis=1)
X_val_drop = X_val.drop(categorical_columns, axis=1)
X_test_drop = X_test_test.drop(categorical_columns, axis=1)

# creating an instance of a model and fitting it
linear_regressor = LinearRegression()
linear_regressor.fit(X_train_drop, y_train)

# getting predictions for test and validation parts of the dataset:
y_test_pred = linear_regressor.predict(X_test_drop)
y_val_pred = linear_regressor.predict(X_val_drop)

# evaluating the model's quality
val_mape = mean_absolute_percentage_error(y_val, y_val_pred)
val_rmse = mean_squared_error(y_val, y_val_pred, squared=False)

test_mape = mean_absolute_percentage_error(y_test_test, y_test_pred)
test_rmse = mean_squared_error(y_test_test, y_test_pred, squared=False)

print(f"MAPE score on validation set: {val_mape}")
print(f"RMSE score on validation set: {val_rmse}")
print(f"MAPE score on test set: {test_mape}")
print(f"RMSE score on test set: {test_rmse}")

# NOTE: a model cannot be fitted without the proper handling of categorical
# features first. In this task, we can either drop them or encode. Since encoding
# is supposed to be performed later, some of the features have been dropped.
# Sklearn's linear regressor cannot perform well without the preliminary data preprocessing.

MAPE score on validation set: 0.6694666794357188
RMSE score on validation set: 59766.824655346245
MAPE score on test set: 0.6514027853443363
RMSE score on test set: 63544.73382433843


## Задание 3 (0.5 балла)

Обучите модель бустинга выберите любую из трех библиотек catboost, xgboost,
lightgbm и оцените её качество через mape и rmse

In [41]:
from xgboost import XGBRegressor

# creating an instance of a model and fitting it
xg_boost = XGBRegressor()
xg_boost.fit(X_train_drop, y_train)

# getting predictions for test and validation parts of the dataset:
y_test_pred = xg_boost.predict(X_test_drop)
y_val_pred = xg_boost.predict(X_val_drop)

# evaluating the model's quality
val_mape = mean_absolute_percentage_error(y_val, y_val_pred)
val_rmse = mean_squared_error(y_val, y_val_pred, squared=False)

test_mape = mean_absolute_percentage_error(y_test_test, y_test_pred)
test_rmse = mean_squared_error(y_test_test, y_test_pred, squared=False)

print(f"MAPE score on validation set: {val_mape}")
print(f"RMSE score on validation set: {val_rmse}")
print(f"MAPE score on test set: {test_mape}")
print(f"RMSE score on test set: {test_rmse}")

MAPE score on validation set: 0.6059462896177483
RMSE score on validation set: 57367.10506318914
MAPE score on test set: 0.6084349615052234
RMSE score on test set: 62991.68013441048


## Задание 4 (0.5 балла)

Далее предобработайте категориальные признаки с помощью onehot кодирования, labelencoder и счетчиков (mean target encoding) сравните качество моделей с помощью mape и rmse. Необходимо обучить две модели линейную и бустинг. Какие выводы можно сделать? Выберите лучшую из моделей и дальше будем работать с ней

In [42]:
# checking if there are any missing values:
df.isnull().sum() # no missing values found

work_year             0
experience_level      0
employment_type       0
job_title             0
salary                0
salary_currency       0
salary_in_usd         0
employee_residence    0
remote_ratio          0
company_location      0
company_size          0
dtype: int64

In [43]:
# checking how many unique values we have for certain features
print(df['employment_type'].unique())
print(df['experience_level'].unique())
print(df['company_size'].unique())

['FT' 'CT' 'FL' 'PT']
['SE' 'MI' 'EN' 'EX']
['L' 'S' 'M']


In [44]:
!pip install category_encoders
from category_encoders import TargetEncoder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [100]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
target_encoder = TargetEncoder()


## Задание 5 (0.5 балла)

Покажите, где ошибается ваша модель. Выведите топ 20 примеров с наибольшей ошибкой. Проанализируете их, постарайтесь выделить группы с причинами ошибок в моделе. Какие выводы можно сделать? Что стоит изменить в данных чтобы улучшить качество модели?

## Задание 6 (0.5 балла)

Придумайте признаки для улучшения качества модели на основе предыдущего пункта. Как вам кажется какими признаками можно улучшить качетсво модели? Реализуйте признаки и проверьте улучшилось ли качество модели. Полный бал за задание ставится даже при отсутствии улучшения качества модели, важно попробовать проверить свои гипотезы и сделать выводы

## Задание 7 (0.5 балла)

Теперь используете исключительно catboost. Обучить модель. Подберите оптимальные гиперпараметры. Используйте pool для передачи данных в модель с указанием какие признаки категориальные, а какие нет с помощью параметры cat_features. Оцените качество итоговой модели, скорость обучения и скорость предсказания

## Задание 8 (0.5 балла)

Теперь используете исключительно xgboost. Обучить модель. Подберите оптимальные гиперпараметры. Закодируйте категориальные переменные, как хотите. Оцените качество итоговой модели, скорость обучения и скорость предсказания.

## Задание 9 (0.5 балла)

Теперь используете исключительно lightgbm. Обучить модель. Подберите оптимальные гиперпараметры. Закодируйте категориальные переменные, как хотите. Оцените качество итоговой модели, скорость обучения и скорость предсказания.

## Задание 10 (0.5 балла)

Сделайте выводы про модели бустинга, какая из моделей показала лучший результат по качеству, скорости обучения и скорости предсказания